In [1]:
%%sql

CREATE DATABASE IF NOT EXISTS cadastro;

25/02/18 13:53:10 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [2]:
%%sql
CREATE TABLE cadastro.usuarios (
    id INT,
    nome STRING,
    endereco STRING,
    email STRING
) USING ICEBERG;


++
||
++
++

In [5]:
%%sql
INSERT INTO cadastro.usuarios VALUES
(1, 'Alice', 'Rua A, 123', 'alice@email.com'),
(2, 'Bruno', 'Rua B, 456', 'bruno@email.com'),
(3, 'Carlos', 'Rua C, 789', 'carlos@email.com'),
(4, 'Daniela', 'Rua D, 101', 'daniela@email.com'),
(5, 'Eduardo', 'Rua E, 202', 'eduardo@email.com'),
(6, 'Fernanda', 'Rua F, 303', 'fernanda@email.com'),
(7, 'Gabriel', 'Rua G, 404', 'gabriel@email.com'),
(8, 'Helena', 'Rua H, 505', 'helena@email.com'),
(9, 'Igor', 'Rua I, 606', 'igor@email.com'),
(10, 'Juliana', 'Rua J, 707', 'juliana@email.com');


++
||
++
++

In [6]:
%%sql
select * from cadastro.usuarios.history

made_current_at,snapshot_id,parent_id,is_current_ancestor
2025-02-18 13:53:37.781000,1819324054691804068,None,True


In [7]:
%%sql
select * from cadastro.usuarios

id,nome,endereco,email
1,Alice,"Rua A, 123",alice@email.com
2,Bruno,"Rua B, 456",bruno@email.com
3,Carlos,"Rua C, 789",carlos@email.com
4,Daniela,"Rua D, 101",daniela@email.com
5,Eduardo,"Rua E, 202",eduardo@email.com
6,Fernanda,"Rua F, 303",fernanda@email.com
7,Gabriel,"Rua G, 404",gabriel@email.com
8,Helena,"Rua H, 505",helena@email.com
9,Igor,"Rua I, 606",igor@email.com
10,Juliana,"Rua J, 707",juliana@email.com


In [8]:
%%sql
DELETE FROM cadastro.usuarios WHERE nome IN ('Carlos', 'Fernanda', 'Igor');


++
||
++
++

In [9]:
%%sql
select * from cadastro.usuarios.history

made_current_at,snapshot_id,parent_id,is_current_ancestor
2025-02-18 13:53:37.781000,1819324054691804068,None,True
2025-02-18 13:54:14.995000,5423887048018366121,1819324054691804068,True


In [10]:
%%sql
SELECT * FROM cadastro.usuarios.snapshots;

committed_at,snapshot_id,parent_id,operation,manifest_list,summary
2025-02-18 13:53:37.781000,1819324054691804068,None,append,s3://warehouse/cadastro/usuarios/metadata/snap-1819324054691804068-1-0dcb2df3-f9b6-416e-9c07-03db75044c1f.avro,"{'spark.app.id': 'local-1739885006778', 'changed-partition-count': '1', 'added-data-files': '5', 'total-equality-deletes': '0', 'added-records': '10', 'total-position-deletes': '0', 'added-files-size': '6017', 'total-delete-files': '0', 'total-files-size': '6017', 'total-records': '10', 'total-data-files': '5'}"
2025-02-18 13:54:14.995000,5423887048018366121,1819324054691804068,overwrite,s3://warehouse/cadastro/usuarios/metadata/snap-5423887048018366121-1-d146e31f-b683-43b3-8e61-f2590029a7a4.avro,"{'added-data-files': '1', 'total-equality-deletes': '0', 'added-records': '3', 'deleted-data-files': '3', 'deleted-records': '6', 'total-records': '7', 'spark.app.id': 'local-1739885006778', 'removed-files-size': '3623', 'changed-partition-count': '1', 'total-position-deletes': '0', 'added-files-size': '1285', 'total-delete-files': '0', 'total-files-size': '3679', 'total-data-files': '3'}"


In [11]:
%%sql
SELECT * FROM cadastro.usuarios FOR VERSION AS OF 1819324054691804068;


id,nome,endereco,email
1,Alice,"Rua A, 123",alice@email.com
2,Bruno,"Rua B, 456",bruno@email.com
3,Carlos,"Rua C, 789",carlos@email.com
4,Daniela,"Rua D, 101",daniela@email.com
5,Eduardo,"Rua E, 202",eduardo@email.com
6,Fernanda,"Rua F, 303",fernanda@email.com
7,Gabriel,"Rua G, 404",gabriel@email.com
8,Helena,"Rua H, 505",helena@email.com
9,Igor,"Rua I, 606",igor@email.com
10,Juliana,"Rua J, 707",juliana@email.com


In [19]:
%%sql
SELECT old.*
FROM cadastro.usuarios FOR VERSION AS OF 1819324054691804068 AS old
LEFT JOIN cadastro.usuarios AS new
ON old.id = new.id
WHERE new.id IS NULL;

id,nome,endereco,email
3,Carlos,"Rua C, 789",carlos@email.com
6,Fernanda,"Rua F, 303",fernanda@email.com
9,Igor,"Rua I, 606",igor@email.com
